In [1]:
from __future__ import division, print_function
from ipywidgets import interact, interactive, fixed, interact_manual    
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy as sp
from scipy import ndimage
from IPython.core.debugger import set_trace
import cv2
import sys
from pprint import pprint
from random import randint
from nuclei.foci import *
import nuclei.girder as g
import nuclei.girder.time_lapse_data as reader

plt.rcParams['image.cmap'] = 'gist_earth'
matplotlib.rcParams['figure.figsize'] = (11.0, 11.0)


    
    

In [2]:
# Hard coded to folder ids
# TODO: Iterate over series in a folder.
# /Gwenda/LawDNADamageFociData
# 5aaf02831fbb9006233ae6a2  073117 BT549_Sv40_Bt549_SV40 NLS GFP 53BP1 mcherry_2017_07_31__18_31_54
#    series000: 5aaf02871fbb9006233ae6a3
#    ...
# 5a95b8361fbb904fdd27d09a  080217 BT549 NLS GFP 53BP1 mcherry_2017_08_02__17_46_03
# 5aaec5d21fbb9006233ad700  080817 BT549 NLS GFP 53BP1 mcherry_2017_08_08__17_27_10

# this function is defined in foci.py
# 073117 BT549_Sv40_Bt549_SV40 NLS GFP 53BP1 mcherry_2017_07_31__18_31_54/series000
# 92 time steps in this series (pattern "time%04d"%idx )
# three shannels are read into a single image (BGR)
# Looks like the brightfield channel is the first (0)
source = reader.time_lapse_data()
# 073117 BT549_Sv40_Bt549_SV40 NLS GFP 53BP1 mcherry_2017_07_31__18_31_54series000
source.load('5ada43f41fbb9005ff7260eb')
num_series = source.get_number_of_series()

#parameter
series_idx = 11


num_time_steps = source.get_series_length(series_idx)
series_folder = source.get_series_folder(series_idx)




# Load the tracks from girder from the two frames.
annotation = g.Annotation("tracks")
annotation.LoadFromItem(before_item_obj['_id'])
before_tracks = annotation.annot_obj['elements']
annotation.LoadFromItem(after_item_obj['_id'])
after_tracks = annotation.annot_obj['elements']


#NEED TO PUT THE CIRCLEROW CODE HERE
#ALSO NEED TO WRITE SOMETHING THAT GETS THE GAP


#loop over the tracks for each nuclei
for track in tracks:
    color = track['lineColor'];
    # convert the color to hex format.
    hex_digits = "0123456789abcdef"
    r1 = hex_digits[int(color[0]/16)]
    r2 = hex_digits[color[0]%16]
    g1 = hex_digits[int(color[1]/16)]
    g2 = hex_digits[color[1]%16]
    b1 = hex_digits[int(color[2]/16)]
    b2 = hex_digits[color[2]%16]
    hex_color = "#"+r1+r2+g1+g2+b1+b2
    
    #FOR ROW 1
    #calulate the time for which the track crosses the row of circular barriers
    beforetime = track.cross_line(row1[0],row1[1])
    
    #make sure the time isn't -1 or the start time
    if(time > track.get_start_time()):
        row = 1
        after_time = before_time + 10
        # load the image objects from girder for the two frames (before and after).
        before_im, before_item_obj = source.get_image(series_idx, before_time)
        after_im, after_item_obj = source.get_image(series_idx, after_time)

        # Get the images for these cells.
        # TODO: Put the time value in the z channel.
        # Assume the last track point is from the image time step.

        # these will not be in the right order.

        before_images = []
        after_images = []
        cross_track = []

        for i in range(10):
            # Assume the last track point is from the image time step.
            pt_idx = len(after_track) - i - 1
            if pt_idx >= 0:
                x,y,z = after_track[pt_idx]
                z = after_time - i
                cross_track.append((x,y,z))
                im, item_obj = source.get_image(series_idx, z)
                im = im[y-64:y+64, x-64:x+64, ...]
                after_images.append(im)

        for i in range(10):
            # Assume the last track point is from the image time step.
            pt_idx = len(before_track) - i - 1
            if pt_idx >= 0:
                x,y,z = before_track[pt_idx]
                z = before_time - i
                cross_track.append((x,y,z))
                im, item_obj = source.get_image(series_idx, before_time-i)
                im = im[y-64:y+64, x-64:x+64, ...]
                before_images.append(im)
                #print((x,y))

        before_images.reverse()
        after_images.reverse()
        cross_track.reverse()


        fig, ax = plt.subplots(4,5, sharey=True, figsize=(12,10))

        g.upload_images(before_images + after_images, 'cross_%d_%d_%d'%(before_time, row, gap), series_folder['_id'])
    
    #FOR ROW 2
    #calulate the time for which the track crosses the row of circular barriers
    beforetime = track.cross_line(row2[0],row2[1])
    
    #make sure the time isn't -1 or the start time
    if(time > track.get_start_time()):
        row = 2
        after_time = before_time + 10
        # load the image objects from girder for the two frames (before and after).
        before_im, before_item_obj = source.get_image(series_idx, before_time)
        after_im, after_item_obj = source.get_image(series_idx, after_time)

        # Get the images for these cells.
        # TODO: Put the time value in the z channel.
        # Assume the last track point is from the image time step.

        # these will not be in the right order.

        before_images = []
        after_images = []
        cross_track = []

        for i in range(10):
            # Assume the last track point is from the image time step.
            pt_idx = len(after_track) - i - 1
            if pt_idx >= 0:
                x,y,z = after_track[pt_idx]
                z = after_time - i
                cross_track.append((x,y,z))
                im, item_obj = source.get_image(series_idx, z)
                im = im[y-64:y+64, x-64:x+64, ...]
                after_images.append(im)

        for i in range(10):
            # Assume the last track point is from the image time step.
            pt_idx = len(before_track) - i - 1
            if pt_idx >= 0:
                x,y,z = before_track[pt_idx]
                z = before_time - i
                cross_track.append((x,y,z))
                im, item_obj = source.get_image(series_idx, before_time-i)
                im = im[y-64:y+64, x-64:x+64, ...]
                before_images.append(im)
                #print((x,y))

        before_images.reverse()
        after_images.reverse()
        cross_track.reverse()


        fig, ax = plt.subplots(4,5, sharey=True, figsize=(12,10))

        g.upload_images(before_images + after_images, 'cross_%d_%d_%d'%(before_time, row, gap), series_folder['_id'])
        
    #FOR ROW 3
    #calulate the time for which the track crosses the row of circular barriers
    beforetime = track.cross_line(row2[0],row2[1])
    
    #make sure the time isn't -1 or the start time
    if(time > track.get_start_time()):
        row = 3
        after_time = before_time + 10
        # load the image objects from girder for the two frames (before and after).
        before_im, before_item_obj = source.get_image(series_idx, before_time)
        after_im, after_item_obj = source.get_image(series_idx, after_time)

        # Get the images for these cells.
        # TODO: Put the time value in the z channel.
        # Assume the last track point is from the image time step.

        # these will not be in the right order.

        before_images = []
        after_images = []
        cross_track = []

        for i in range(10):
            # Assume the last track point is from the image time step.
            pt_idx = len(after_track) - i - 1
            if pt_idx >= 0:
                x,y,z = after_track[pt_idx]
                z = after_time - i
                cross_track.append((x,y,z))
                im, item_obj = source.get_image(series_idx, z)
                im = im[y-64:y+64, x-64:x+64, ...]
                after_images.append(im)

        for i in range(10):
            # Assume the last track point is from the image time step.
            pt_idx = len(before_track) - i - 1
            if pt_idx >= 0:
                x,y,z = before_track[pt_idx]
                z = before_time - i
                cross_track.append((x,y,z))
                im, item_obj = source.get_image(series_idx, before_time-i)
                im = im[y-64:y+64, x-64:x+64, ...]
                before_images.append(im)
                #print((x,y))

        before_images.reverse()
        after_images.reverse()
        cross_track.reverse()


        fig, ax = plt.subplots(4,5, sharey=True, figsize=(12,10))

        g.upload_images(before_images + after_images, 'cross_%d_%d_%d'%(before_time, row, gap), series_folder['_id'])



NameError: name 'before_item_obj' is not defined